In [ ]:
def to_integral_img(img):
    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            new_val = img[row, col]
            if row != 0:
                new_val += img[row - 1, col]
            if col != 0:
                new_val += img[row, col - 1]
            if row != 0 and col != 0:
                new_val -= img[row - 1, col - 1]
            img[row, col] = new_val
    return img

